In [1]:
# =============================================================================
# QB DECISION TRAINER - STREAMLINED NOTEBOOK
# =============================================================================
# This notebook runs the coaching app using pre-exported data files.
# Required files in DATA_PATH:
#   - receivers_app.parquet
#   - plays_app.parquet
#   - tracking_app.parquet
#   - supplementary_app.parquet
# =============================================================================

# %% [markdown]
# # Cell 1: Imports

# %%
import pandas as pd
import numpy as np
from dash import Dash, html, dcc, callback, Output, Input, State
import plotly.graph_objects as go

In [2]:
# %% [markdown]
# # Cell 2: Load Data

# %%
# Path to exported app data files
DATA_PATH = r'C:\Users\carso\Downloads\nfl-big-data-bowl-2026-analytics'

print("Loading data files...")
print("="*70)

# Load all files
receivers_app = pd.read_parquet(f'{DATA_PATH}/receivers_app.parquet')
plays_app = pd.read_parquet(f'{DATA_PATH}/plays_app.parquet')
tracking_app = pd.read_parquet(f'{DATA_PATH}/tracking_app.parquet')
supplementary_app = pd.read_parquet(f'{DATA_PATH}/supplementary_app.parquet')

print(f"✓ receivers_app: {len(receivers_app):,} rows")
print(f"✓ plays_app: {len(plays_app):,} rows")
print(f"✓ tracking_app: {len(tracking_app):,} rows")
print(f"✓ supplementary_app: {len(supplementary_app):,} rows")

# Split tracking into input/output for easier access
input_data = tracking_app[tracking_app['source'] == 'input'].copy()
output_data = tracking_app[tracking_app['source'] == 'output'].copy()

print(f"\n  Input tracking frames: {len(input_data):,}")
print(f"  Output tracking frames: {len(output_data):,}")


Loading data files...
✓ receivers_app: 63,219 rows
✓ plays_app: 13,770 rows
✓ tracking_app: 5,295,049 rows
✓ supplementary_app: 18,009 rows

  Input tracking frames: 4,754,317
  Output tracking frames: 540,732


In [3]:
# %% [markdown]
# # Cell 3: Feature Summary

# %%
def print_feature_summary(df, name):
    """Print detailed feature summary for a dataframe."""
    print(f"\n{'='*70}")
    print(f"FEATURE SUMMARY: {name}")
    print(f"{'='*70}")
    print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
    print(f"Memory: {df.memory_usage(deep=True).sum() / 1e6:.2f} MB")
    print("-"*70)
    
    for col in df.columns:
        dtype = df[col].dtype
        non_null = df[col].notna().sum()
        null_pct = (1 - non_null / len(df)) * 100
        
        print(f"\n{col}")
        print(f"  dtype: {dtype}")
        print(f"  non-null: {non_null:,} ({100-null_pct:.1f}%)")
        
        if dtype == 'object' or str(dtype) == 'category':
            # Categorical - show unique values
            unique_vals = df[col].dropna().unique()
            n_unique = len(unique_vals)
            print(f"  unique: {n_unique}")
            if n_unique <= 15:
                vals_str = ', '.join([str(v) for v in sorted(unique_vals)[:15]])
                print(f"  values: {vals_str}")
            else:
                sample = ', '.join([str(v) for v in list(unique_vals)[:10]])
                print(f"  sample: {sample}...")
        elif dtype in ['int64', 'int32', 'float64', 'float32']:
            # Numeric - show min/max
            print(f"  min: {df[col].min()}")
            print(f"  max: {df[col].max()}")
        elif dtype == 'bool':
            true_count = df[col].sum()
            print(f"  True: {true_count:,} ({true_count/len(df)*100:.1f}%)")
            print(f"  False: {len(df) - true_count:,} ({(len(df)-true_count)/len(df)*100:.1f}%)")

# Print summaries for all dataframes
print_feature_summary(receivers_app, "receivers_app")
print_feature_summary(plays_app, "plays_app")
print_feature_summary(tracking_app, "tracking_app")
print_feature_summary(supplementary_app, "supplementary_app")


FEATURE SUMMARY: receivers_app
Shape: 63,219 rows × 17 columns
Memory: 20.87 MB
----------------------------------------------------------------------

game_id
  dtype: int64
  non-null: 63,219 (100.0%)
  min: 2023090700
  max: 2024010713

play_id
  dtype: int64
  non-null: 63,219 (100.0%)
  min: 54
  max: 5258

nfl_id
  dtype: int64
  non-null: 63,219 (100.0%)
  min: 30842
  max: 56663

receiver_name
  dtype: object
  non-null: 63,219 (100.0%)
  unique: 516
  sample: Josh Reynolds, Amon-Ra St. Brown, Brock Wright, Jerick McKinnon, Justin Watson, Marquez Valdes-Scantling, Noah Gray, Skyy Moore, Richie James, Clyde Edwards-Helaire...

receiver_position
  dtype: object
  non-null: 63,219 (100.0%)
  unique: 12
  values: CB, DT, FB, ILB, K, MLB, P, QB, RB, T, TE, WR

predicted_route
  dtype: object
  non-null: 63,219 (100.0%)
  unique: 12
  values: ANGLE, CORNER, CROSS, FLAT, GO, HITCH, IN, OUT, POST, SCREEN, SLANT, WHEEL

catch_probability
  dtype: float64
  non-null: 63,219 (100.0%)
  m

In [4]:
# %% [markdown]
# # Cell 4: Play Loader Function

# %%
def load_play_for_coaching(game_id, play_id):
    """
    Load all data needed for a single play in the coaching app.
    
    Returns dict with:
        - predictions: DataFrame of receiver predictions
        - play_info: Series with play metadata
        - input_frames: sorted list of input frame IDs
        - output_frames: sorted list of output frame IDs
        - best_option: dict with best receiver info
        - actual_target: dict with actual target info (or None)
    """
    # Get receiver predictions for this play
    preds = receivers_app[
        (receivers_app['game_id'] == game_id) & 
        (receivers_app['play_id'] == play_id)
    ].copy()
    
    if len(preds) == 0:
        return None
    
    # Get play metadata
    play_info = plays_app[
        (plays_app['game_id'] == game_id) & 
        (plays_app['play_id'] == play_id)
    ]
    
    if len(play_info) == 0:
        return None
    
    play_info = play_info.iloc[0]
    
    # Get frame lists
    input_frames = sorted(input_data[
        (input_data['game_id'] == game_id) & 
        (input_data['play_id'] == play_id)
    ]['frame_id'].unique().tolist())
    
    output_frames = sorted(output_data[
        (output_data['game_id'] == game_id) & 
        (output_data['play_id'] == play_id)
    ]['frame_id'].unique().tolist())
    
    if len(input_frames) == 0:
        return None
    
    # Best option
    best = preds[preds['is_best_option'] == True]
    if len(best) == 0:
        best = preds.loc[preds['catch_probability'].idxmax()]
    else:
        best = best.iloc[0]
    
    best_option = {
        'nfl_id': int(best['nfl_id']),
        'receiver_name': best.get('receiver_name', 'Unknown'),
        'receiver_position': best.get('receiver_position', '??'),
        'predicted_route': best.get('predicted_route', '?'),
        'catch_probability': float(best['catch_probability'])
    }
    
    # Actual target - uses is_targeted column (not was_targeted)
    actual_target = None
    targeted = preds[preds['is_targeted'] == True]
    if len(targeted) > 0:
        t = targeted.iloc[0]
        actual_target = {
            'nfl_id': int(t['nfl_id']),
            'receiver_name': t.get('receiver_name', 'Unknown'),
            'receiver_position': t.get('receiver_position', '??'),
            'predicted_route': t.get('predicted_route', '?'),
            'catch_probability': float(t['catch_probability'])
        }
    
    # Calculate pause frame (capped at max available)
    timing_frames = int(play_info['timing_frames'])
    pause_idx = min(timing_frames, len(input_frames) - 1)
    pause_frame = input_frames[pause_idx]
    
    # Get QB position at first frame
    first_frame = input_frames[0]
    qb_data = input_data[
        (input_data['game_id'] == game_id) & 
        (input_data['play_id'] == play_id) &
        (input_data['frame_id'] == first_frame) &
        (input_data['player_role'] == 'Passer')
    ]
    
    if len(qb_data) > 0:
        qb_pos = [float(qb_data['x'].iloc[0]), float(qb_data['y'].iloc[0])]
    else:
        qb_pos = [25.0, 26.65]  # Default center field
    
    # Receiver rankings by season catches
    preds_sorted = preds.sort_values('total_catches_season', ascending=False)
    receiver_rankings = {
        int(row['nfl_id']): idx + 1 
        for idx, (_, row) in enumerate(preds_sorted.iterrows())
    }
    
    # Get result from play_info - uses caught column (1/0)
    # Convert to result string for display
    result = play_info.get('play_result', 'UNKNOWN')
    
    return {
        'game_id': game_id,
        'play_id': play_id,
        'predictions': preds,
        'play_info': play_info,
        'concept': play_info.get('concept_final', 'UNKNOWN'),
        'timing_frames': timing_frames,
        'pause_frame': pause_frame,
        'input_frames': input_frames,
        'output_frames': output_frames,
        'best_option': best_option,
        'actual_target': actual_target,
        'result': result,
        'qb_pos': qb_pos,
        'receiver_rankings': receiver_rankings,
        'receiver_ids': preds['nfl_id'].tolist()
    }

# Test the loader
print("\nTesting play loader...")
sample = plays_app.sample(1).iloc[0]
test_play = load_play_for_coaching(sample['game_id'], sample['play_id'])
if test_play:
    print(f"✓ Loaded play {sample['game_id']}/{sample['play_id']}")
    print(f"  Concept: {test_play['concept']}")
    print(f"  Receivers: {len(test_play['predictions'])}")
    print(f"  Input frames: {len(test_play['input_frames'])}")
    print(f"  Best option: {test_play['best_option']['receiver_name']} ({test_play['best_option']['catch_probability']:.1%})")
else:
    print("✗ Failed to load test play")


Testing play loader...
✓ Loaded play 2023110501/3784
  Concept: DEEP-PA
  Receivers: 3
  Input frames: 24
  Best option: KhaDarel Hodge (66.5%)


In [5]:
# =============================================================================
# QB DECISION TRAINER - STREAMLINED NOTEBOOK
# =============================================================================
# This notebook runs the coaching app using pre-exported data files.
# Required files in DATA_PATH:
#   - receivers_app.parquet
#   - plays_app.parquet
#   - tracking_app.parquet
#   - supplementary_app.parquet
# =============================================================================

# %% [markdown]
# # Cell 1: Imports

# %%
import pandas as pd
import numpy as np
from dash import Dash, html, dcc, callback, Output, Input, State
import plotly.graph_objects as go

# %% [markdown]
# # Cell 2: Load Data

# %%
# Path to exported app data files
DATA_PATH = r'C:\Users\carso\Downloads\nfl-big-data-bowl-2026-analytics'

print("Loading data files...")
print("="*70)

# Load all files
receivers_app = pd.read_parquet(f'{DATA_PATH}/receivers_app.parquet')
plays_app = pd.read_parquet(f'{DATA_PATH}/plays_app.parquet')
tracking_app = pd.read_parquet(f'{DATA_PATH}/tracking_app.parquet')
supplementary_app = pd.read_parquet(f'{DATA_PATH}/supplementary_app.parquet')

print(f"✓ receivers_app: {len(receivers_app):,} rows")
print(f"✓ plays_app: {len(plays_app):,} rows")
print(f"✓ tracking_app: {len(tracking_app):,} rows")
print(f"✓ supplementary_app: {len(supplementary_app):,} rows")

# Split tracking into input/output for easier access
input_data = tracking_app[tracking_app['source'] == 'input'].copy()
output_data = tracking_app[tracking_app['source'] == 'output'].copy()

print(f"\n  Input tracking frames: {len(input_data):,}")
print(f"  Output tracking frames: {len(output_data):,}")

# %% [markdown]
# # Cell 3: Feature Summary

# %%
def print_feature_summary(df, name):
    """Print detailed feature summary for a dataframe."""
    print(f"\n{'='*70}")
    print(f"FEATURE SUMMARY: {name}")
    print(f"{'='*70}")
    print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
    print(f"Memory: {df.memory_usage(deep=True).sum() / 1e6:.2f} MB")
    print("-"*70)
    
    for col in df.columns:
        dtype = df[col].dtype
        non_null = df[col].notna().sum()
        null_pct = (1 - non_null / len(df)) * 100
        
        print(f"\n{col}")
        print(f"  dtype: {dtype}")
        print(f"  non-null: {non_null:,} ({100-null_pct:.1f}%)")
        
        if dtype == 'object' or str(dtype) == 'category':
            # Categorical - show unique values
            unique_vals = df[col].dropna().unique()
            n_unique = len(unique_vals)
            print(f"  unique: {n_unique}")
            if n_unique <= 15:
                vals_str = ', '.join([str(v) for v in sorted(unique_vals)[:15]])
                print(f"  values: {vals_str}")
            else:
                sample = ', '.join([str(v) for v in list(unique_vals)[:10]])
                print(f"  sample: {sample}...")
        elif dtype in ['int64', 'int32', 'float64', 'float32']:
            # Numeric - show min/max
            print(f"  min: {df[col].min()}")
            print(f"  max: {df[col].max()}")
        elif dtype == 'bool':
            true_count = df[col].sum()
            print(f"  True: {true_count:,} ({true_count/len(df)*100:.1f}%)")
            print(f"  False: {len(df) - true_count:,} ({(len(df)-true_count)/len(df)*100:.1f}%)")

# Print summaries for all dataframes
print_feature_summary(receivers_app, "receivers_app")
print_feature_summary(plays_app, "plays_app")
print_feature_summary(tracking_app, "tracking_app")
print_feature_summary(supplementary_app, "supplementary_app")

# %% [markdown]
# # Cell 4: Play Loader Function

# %%
def load_play_for_coaching(game_id, play_id):
    """
    Load all data needed for a single play in the coaching app.
    
    Returns dict with:
        - predictions: DataFrame of receiver predictions
        - play_info: Series with play metadata
        - input_frames: sorted list of input frame IDs
        - output_frames: sorted list of output frame IDs
        - best_option: dict with best receiver info
        - actual_target: dict with actual target info (or None)
    """
    # Get receiver predictions for this play
    preds = receivers_app[
        (receivers_app['game_id'] == game_id) & 
        (receivers_app['play_id'] == play_id)
    ].copy()
    
    if len(preds) == 0:
        return None
    
    # Get play metadata
    play_info = plays_app[
        (plays_app['game_id'] == game_id) & 
        (plays_app['play_id'] == play_id)
    ]
    
    if len(play_info) == 0:
        return None
    
    play_info = play_info.iloc[0]
    
    # Get frame lists
    input_frames = sorted(input_data[
        (input_data['game_id'] == game_id) & 
        (input_data['play_id'] == play_id)
    ]['frame_id'].unique().tolist())
    
    output_frames = sorted(output_data[
        (output_data['game_id'] == game_id) & 
        (output_data['play_id'] == play_id)
    ]['frame_id'].unique().tolist())
    
    if len(input_frames) == 0:
        return None
    
    # Best option
    best = preds[preds['is_best_option'] == True]
    if len(best) == 0:
        best = preds.loc[preds['catch_probability'].idxmax()]
    else:
        best = best.iloc[0]
    
    best_option = {
        'nfl_id': int(best['nfl_id']),
        'receiver_name': best.get('receiver_name', 'Unknown'),
        'receiver_position': best.get('receiver_position', '??'),
        'predicted_route': best.get('predicted_route', '?'),
        'catch_probability': float(best['catch_probability'])
    }
    
    # Actual target - uses is_targeted column (not was_targeted)
    actual_target = None
    targeted = preds[preds['is_targeted'] == True]
    if len(targeted) > 0:
        t = targeted.iloc[0]
        actual_target = {
            'nfl_id': int(t['nfl_id']),
            'receiver_name': t.get('receiver_name', 'Unknown'),
            'receiver_position': t.get('receiver_position', '??'),
            'predicted_route': t.get('predicted_route', '?'),
            'catch_probability': float(t['catch_probability'])
        }
    
    # Calculate pause frame (capped at max available)
    timing_frames = int(play_info['timing_frames'])
    pause_idx = min(timing_frames, len(input_frames) - 1)
    pause_frame = input_frames[pause_idx]
    
    # Get QB position at first frame
    first_frame = input_frames[0]
    qb_data = input_data[
        (input_data['game_id'] == game_id) & 
        (input_data['play_id'] == play_id) &
        (input_data['frame_id'] == first_frame) &
        (input_data['player_role'] == 'Passer')
    ]
    
    if len(qb_data) > 0:
        qb_pos = [float(qb_data['x'].iloc[0]), float(qb_data['y'].iloc[0])]
    else:
        qb_pos = [25.0, 26.65]  # Default center field
    
    # Receiver rankings by season catches
    preds_sorted = preds.sort_values('total_catches_season', ascending=False)
    receiver_rankings = {
        int(row['nfl_id']): idx + 1 
        for idx, (_, row) in enumerate(preds_sorted.iterrows())
    }
    
    # Get result from play_info - uses caught column (1/0)
    # Convert to result string for display
    result = play_info.get('play_result', 'UNKNOWN')
    
    return {
        'game_id': game_id,
        'play_id': play_id,
        'predictions': preds,
        'play_info': play_info,
        'concept': play_info.get('concept_final', 'UNKNOWN'),
        'timing_frames': timing_frames,
        'pause_frame': pause_frame,
        'input_frames': input_frames,
        'output_frames': output_frames,
        'best_option': best_option,
        'actual_target': actual_target,
        'result': result,
        'qb_pos': qb_pos,
        'receiver_rankings': receiver_rankings,
        'receiver_ids': preds['nfl_id'].tolist()
    }

# Test the loader
print("\nTesting play loader...")
sample = plays_app.sample(1).iloc[0]
test_play = load_play_for_coaching(sample['game_id'], sample['play_id'])
if test_play:
    print(f"✓ Loaded play {sample['game_id']}/{sample['play_id']}")
    print(f"  Concept: {test_play['concept']}")
    print(f"  Receivers: {len(test_play['predictions'])}")
    print(f"  Input frames: {len(test_play['input_frames'])}")
    print(f"  Best option: {test_play['best_option']['receiver_name']} ({test_play['best_option']['catch_probability']:.1%})")
else:
    print("✗ Failed to load test play")

# %% [markdown]
# # Cell 5: Dash App

# %%
def create_app():
    """Create and return a fresh Dash app instance."""
    
    # Football shape SVG path
    FOOTBALL_PATH = "M 0 -1 C 0.6 -1 1 -0.3 1 0 C 1 0.3 0.6 1 0 1 C -0.6 1 -1 0.3 -1 0 C -1 -0.3 -0.6 -1 0 -1 Z"
    
    # Position colors
    POSITION_COLORS = {
        'WR': '#2196f3',  # Blue
        'TE': '#ffd700',  # Yellow/Gold
        'RB': '#4caf50',  # Green
        'FB': '#4caf50',  # Green
    }
    DEFAULT_COLOR = '#9e9e9e'  # Gray
    
    def create_field_figure():
        """Create empty football field figure."""
        fig = go.Figure()
        
        field_width = 53.3
        field_length = 120
        
        # Green background
        fig.add_shape(type="rect", x0=0, y0=0, x1=field_length, y1=field_width,
                      fillcolor="#2e7d32", line=dict(width=0), layer="below")
        
        # End zones
        fig.add_shape(type="rect", x0=0, y0=0, x1=10, y1=field_width,
                      fillcolor="#1b5e20", line=dict(color="white", width=2), layer="below")
        fig.add_shape(type="rect", x0=110, y0=0, x1=120, y1=field_width,
                      fillcolor="#1b5e20", line=dict(color="white", width=2), layer="below")
        
        # Yard lines
        for yard in range(10, 111, 5):
            width = 2 if yard % 10 == 0 else 1
            fig.add_shape(type="line", x0=yard, y0=0, x1=yard, y1=field_width,
                          line=dict(color="white", width=width), layer="below")
        
        # Yard numbers
        for yard in range(10, 100, 10):
            display_num = yard if yard <= 50 else 100 - yard
            fig.add_annotation(x=yard+10, y=field_width/2, text=str(display_num),
                              font=dict(size=20, color="white"), showarrow=False, opacity=0.5)
        
        fig.update_layout(
            xaxis=dict(range=[-5, 125], showgrid=False, zeroline=False, showticklabels=False),
            yaxis=dict(range=[-5, field_width+5], showgrid=False, zeroline=False, showticklabels=False, scaleanchor="x"),
            plot_bgcolor='rgba(0,0,0,0)',
            paper_bgcolor='#1a1a1a',
            margin=dict(l=20, r=20, t=60, b=20),
            height=500,
            showlegend=True
        )
        
        return fig
    
    def add_players_to_figure(fig, tracking_frame, predictions, show_probs=False, 
                              selected_id=None, selection_correct=None,
                              show_ball=False, ball_pos=None, ball_result=None,
                              actual_receiver_id=None, receiver_rankings=None):
        """Add player positions to figure for a single frame."""
        
        receiver_info = {row['nfl_id']: row for _, row in predictions.iterrows()}
        receiver_ids = set(receiver_info.keys())
        
        offense = tracking_frame[tracking_frame['player_side'] == 'Offense']
        defense = tracking_frame[tracking_frame['player_side'] == 'Defense']
        
        # Plot defenders (red, smaller)
        if len(defense) > 0:
            fig.add_trace(go.Scatter(
                x=defense['x'], y=defense['y'],
                mode='markers',
                marker=dict(size=10, color='#ef5350', line=dict(width=1, color='white')),
                name='Defense',
                hoverinfo='skip',
                showlegend=True
            ))
        
        # Plot offensive players
        for _, player in offense.iterrows():
            nfl_id = player['nfl_id']
            player_role = player.get('player_role', '')
            
            # QB - white square
            if player_role == 'Passer':
                fig.add_trace(go.Scatter(
                    x=[player['x']], y=[player['y']],
                    mode='markers',
                    marker=dict(size=14, color='white', symbol='square', 
                               line=dict(width=2, color='#333')),
                    name='QB',
                    hoverinfo='skip',
                    showlegend=True
                ))
            
            # Receivers
            elif nfl_id in receiver_ids:
                pred = receiver_info[nfl_id]
                prob = pred['catch_probability']
                is_best = pred.get('is_best_option', False)
                route = pred.get('predicted_route', '?')
                position = pred.get('receiver_position', '??')
                receiver_name = pred.get('receiver_name', 'Unknown')
                
                # Get ranking number
                rank_num = receiver_rankings.get(nfl_id, 99) if receiver_rankings else 99
                
                # Format name as "F. LastName"
                name_parts = str(receiver_name).split() if receiver_name else ['?']
                if len(name_parts) >= 2:
                    display_name = f"{name_parts[0][0]}. {name_parts[-1]}"
                else:
                    display_name = str(receiver_name)
                
                # Base color by position
                base_color = POSITION_COLORS.get(position, DEFAULT_COLOR)
                
                # Override color for selection/result states
                if selected_id is not None and nfl_id == selected_id:
                    color = '#4caf50' if selection_correct else '#f44336'
                    line_color = 'white'
                    line_width = 4
                elif show_ball and actual_receiver_id is not None and nfl_id == actual_receiver_id:
                    color = '#4caf50' if ball_result == 'CAUGHT' else '#f44336'
                    line_color = 'white'
                    line_width = 3
                elif is_best and show_probs:
                    color = base_color
                    line_color = '#ffd700'  # Gold outline for best option
                    line_width = 4
                else:
                    color = base_color
                    line_color = 'white'
                    line_width = 1
                
                # Hover text
                if show_probs:
                    hover = f"{display_name}<br>{route}<br>{prob*100:.0f}%"
                else:
                    hover = f"{display_name}<br>{position}"
                
                # Legend name with ranking
                legend_name = f"{rank_num}. {display_name} ({position})"
                
                fig.add_trace(go.Scatter(
                    x=[player['x']], y=[player['y']],
                    mode='markers+text',
                    marker=dict(size=16, color=color, 
                               line=dict(width=line_width, color=line_color)),
                    text=str(rank_num) if rank_num <= 5 else '',
                    textposition='middle center',
                    textfont=dict(size=9, color='white', family='Arial Black'),
                    name=legend_name,
                    customdata=[nfl_id],
                    hovertemplate=hover + "<extra></extra>",
                    showlegend=True
                ))
            
            # Other offense (OL, etc)
            else:
                fig.add_trace(go.Scatter(
                    x=[player['x']], y=[player['y']],
                    mode='markers',
                    marker=dict(size=8, color='#607d8b', line=dict(width=1, color='white')),
                    hoverinfo='skip',
                    showlegend=False
                ))
        
        # Draw football if needed
        if show_ball and ball_pos is not None:
            fig.add_trace(go.Scatter(
                x=[ball_pos[0]], y=[ball_pos[1]],
                mode='markers',
                marker=dict(
                    size=15,
                    color='#8B4513',
                    symbol='path://' + FOOTBALL_PATH,
                    line=dict(width=1, color='white'),
                    angle=45
                ),
                hoverinfo='skip',
                showlegend=False
            ))
        
        return fig
    
    # Create Dash app
    app = Dash(__name__)
    
    app.layout = html.Div([
        # Header
        html.H1("QB Decision Trainer", style={'textAlign': 'center', 'color': 'white'}),
        
        # Play info
        html.Div(id='play-info', style={'textAlign': 'center', 'color': 'white', 'fontSize': '18px', 'marginBottom': '10px'}),
        
        # Field
        dcc.Graph(id='field', config={'displayModeBar': False}),
        
        # Controls
        html.Div([
            html.Button('▶ Play', id='play-btn', n_clicks=0, 
                       style={'fontSize': '20px', 'padding': '10px 30px', 'marginRight': '10px'}),
            html.Button('Next Play', id='next-btn', n_clicks=0,
                       style={'fontSize': '16px', 'padding': '10px 20px'}),
            html.Span(id='frame-display', style={'color': 'white', 'marginLeft': '20px', 'fontSize': '16px'})
        ], style={'textAlign': 'center', 'marginTop': '10px'}),
        
        # Feedback panel
        html.Div(id='feedback-panel', style={'display': 'none'}),
        
        # Stores
        dcc.Store(id='play-data'),
        dcc.Store(id='current-frame', data=1),
        dcc.Store(id='app-phase', data='initial'),
        dcc.Store(id='user-selection'),
        dcc.Store(id='selection-correct'),
        dcc.Store(id='ball-progress', data=0),
        
        # Animation interval
        dcc.Interval(id='animation-interval', interval=100, disabled=True)
        
    ], style={'backgroundColor': '#1a1a1a', 'minHeight': '100vh', 'padding': '20px'})
    
    # Callback: Load new play
    @app.callback(
        Output('play-data', 'data'),
        Output('play-info', 'children'),
        Output('current-frame', 'data'),
        Output('app-phase', 'data'),
        Output('user-selection', 'data'),
        Output('selection-correct', 'data'),
        Output('feedback-panel', 'style'),
        Input('next-btn', 'n_clicks'),
        prevent_initial_call='initial_duplicate'
    )
    def load_new_play(n_clicks):
        """Load a random play."""
        sample = plays_app.sample(1).iloc[0]
        game_id = int(sample['game_id'])
        play_id = int(sample['play_id'])
        
        play = load_play_for_coaching(game_id, play_id)
        
        if play is None:
            return None, "Error loading play", 1, 'initial', None, None, {'display': 'none'}
        
        play_json = {
            'game_id': game_id,
            'play_id': play_id,
            'concept': play['concept'],
            'timing_frames': play['timing_frames'],
            'pause_frame': play['pause_frame'],
            'input_frames': play['input_frames'],
            'output_frames': play['output_frames'],
            'receiver_ids': [int(x) for x in play['receiver_ids']],
            'best_nfl_id': play['best_option']['nfl_id'],
            'best_route': play['best_option']['predicted_route'],
            'best_prob': play['best_option']['catch_probability'],
            'result': play['result'],
            'qb_pos': play['qb_pos'],
            'receiver_rankings': {int(k): int(v) for k, v in play['receiver_rankings'].items()}
        }
        
        if play['actual_target'] is not None:
            play_json['actual_nfl_id'] = play['actual_target']['nfl_id']
            play_json['actual_route'] = play['actual_target']['predicted_route']
            play_json['actual_prob'] = play['actual_target']['catch_probability']
        
        info = f"Game {game_id} | Play {play_id} | {play['concept']} | Decide in {play['timing_frames']/10:.1f}s"
        start_frame = play['input_frames'][0] if play['input_frames'] else 1
        
        return play_json, info, start_frame, 'initial', None, None, {'display': 'none'}
    
    # Callback: Update field
    @app.callback(
        Output('field', 'figure'),
        Output('frame-display', 'children'),
        Input('current-frame', 'data'),
        Input('play-data', 'data'),
        Input('app-phase', 'data'),
        Input('user-selection', 'data'),
        Input('selection-correct', 'data'),
        Input('ball-progress', 'data'),
    )
    def update_field(current_frame, play_data, phase, selected_id, selection_correct, ball_progress):
        """Update field visualization."""
        if play_data is None:
            fig = create_field_figure()
            return fig, "Load a play to begin"
        
        game_id = play_data['game_id']
        play_id = play_data['play_id']
        input_frames = play_data.get('input_frames', [])
        output_frames = play_data.get('output_frames', [])
        
        # Get tracking data
        tracking_frame = input_data[
            (input_data['game_id'] == game_id) & 
            (input_data['play_id'] == play_id) &
            (input_data['frame_id'] == current_frame)
        ]
        
        if len(tracking_frame) == 0:
            tracking_frame = output_data[
                (output_data['game_id'] == game_id) & 
                (output_data['play_id'] == play_id) &
                (output_data['frame_id'] == current_frame)
            ]
        
        if len(tracking_frame) == 0:
            all_frames = sorted(set(input_frames + output_frames))
            if all_frames:
                closest = min(all_frames, key=lambda x: abs(x - current_frame))
                tracking_frame = input_data[
                    (input_data['game_id'] == game_id) & 
                    (input_data['play_id'] == play_id) &
                    (input_data['frame_id'] == closest)
                ]
                if len(tracking_frame) == 0:
                    tracking_frame = output_data[
                        (output_data['game_id'] == game_id) & 
                        (output_data['play_id'] == play_id) &
                        (output_data['frame_id'] == closest)
                    ]
        
        preds = receivers_app[
            (receivers_app['game_id'] == game_id) & 
            (receivers_app['play_id'] == play_id)
        ]
        
        fig = create_field_figure()
        
        show_probs = phase in ['paused', 'selected']
        show_ball = phase == 'post-throw' and ball_progress > 0
        
        ball_pos = None
        if show_ball and 'actual_nfl_id' in play_data:
            qb_pos = play_data['qb_pos']
            actual_rec = tracking_frame[tracking_frame['nfl_id'] == play_data['actual_nfl_id']]
            if len(actual_rec) > 0:
                rec_x, rec_y = float(actual_rec['x'].iloc[0]), float(actual_rec['y'].iloc[0])
                ball_pos = [
                    qb_pos[0] + (rec_x - qb_pos[0]) * ball_progress,
                    qb_pos[1] + (rec_y - qb_pos[1]) * ball_progress
                ]
        
        actual_receiver_id = play_data.get('actual_nfl_id') if phase == 'post-throw' else None
        receiver_rankings = play_data.get('receiver_rankings', {})
        
        if len(tracking_frame) > 0:
            fig = add_players_to_figure(
                fig, tracking_frame, preds, 
                show_probs=show_probs,
                selected_id=selected_id,
                selection_correct=selection_correct,
                show_ball=show_ball,
                ball_pos=ball_pos,
                ball_result=play_data.get('result'),
                actual_receiver_id=actual_receiver_id,
                receiver_rankings=receiver_rankings
            )
        
        fig.update_layout(
            legend=dict(
                bgcolor='rgba(30,30,30,0.9)',
                bordercolor='white',
                borderwidth=1,
                font=dict(color='white', size=11),
                x=1.02, y=1, xanchor='left'
            )
        )
        
        # Status text
        if phase == 'initial':
            status = "Press Play to start"
        elif phase == 'running':
            status = f"Frame {current_frame}"
        elif phase == 'paused':
            status = "🎯 SELECT A RECEIVER"
        elif phase == 'selected':
            status = "Selection made - Press Play to continue"
        elif phase == 'post-throw':
            if ball_progress >= 1.0:
                status = f"{'✅ COMPLETE' if play_data.get('result') == 'CAUGHT' else '❌ INCOMPLETE'}"
            else:
                status = "Ball in the air..."
        else:
            status = ""
        
        fig.update_layout(title=dict(text=status, font=dict(color='white', size=16)))
        
        max_frame = max(input_frames + output_frames) if (input_frames or output_frames) else current_frame
        frame_text = f"Frame {current_frame} / {max_frame}"
        
        return fig, frame_text
    
    # Callback: Toggle play
    @app.callback(
        Output('animation-interval', 'disabled'),
        Output('app-phase', 'data', allow_duplicate=True),
        Output('current-frame', 'data', allow_duplicate=True),
        Output('ball-progress', 'data', allow_duplicate=True),
        Input('play-btn', 'n_clicks'),
        State('app-phase', 'data'),
        State('play-data', 'data'),
        State('current-frame', 'data'),
        prevent_initial_call=True
    )
    def toggle_play(n_clicks, phase, play_data, current_frame):
        """Handle play button."""
        if play_data is None:
            return True, 'initial', 1, 0
        
        input_frames = play_data.get('input_frames', [])
        first_frame = input_frames[0] if input_frames else 1
        
        if phase == 'initial':
            return False, 'running', first_frame, 0
        elif phase == 'paused':
            return True, 'paused', current_frame, 0
        elif phase == 'selected':
            return False, 'post-throw', current_frame, 0
        elif phase == 'running':
            return True, 'paused', current_frame, 0
        elif phase == 'post-throw':
            return False, 'post-throw', current_frame, 0
        
        return True, phase, current_frame, 0
    
    # Callback: Advance frame
    @app.callback(
        Output('current-frame', 'data', allow_duplicate=True),
        Output('animation-interval', 'disabled', allow_duplicate=True),
        Output('app-phase', 'data', allow_duplicate=True),
        Output('ball-progress', 'data', allow_duplicate=True),
        Input('animation-interval', 'n_intervals'),
        State('current-frame', 'data'),
        State('play-data', 'data'),
        State('app-phase', 'data'),
        State('ball-progress', 'data'),
        prevent_initial_call=True
    )
    def advance_frame(n_intervals, current_frame, play_data, phase, ball_progress):
        """Advance animation frame."""
        if play_data is None:
            return 1, True, 'initial', 0
        
        pause_frame = play_data['pause_frame']
        input_frames = play_data.get('input_frames', [])
        output_frames = play_data.get('output_frames', [])
        all_frames = sorted(set(input_frames + output_frames))
        
        if phase == 'running':
            try:
                current_idx = input_frames.index(current_frame)
                next_idx = current_idx + 1
            except ValueError:
                next_idx = 0
                for i, f in enumerate(input_frames):
                    if f > current_frame:
                        next_idx = i
                        break
            
            if next_idx >= len(input_frames):
                return pause_frame, True, 'paused', 0
            
            next_frame = input_frames[next_idx]
            
            if next_frame >= pause_frame:
                return pause_frame, True, 'paused', 0
            
            return next_frame, False, 'running', 0
        
        elif phase == 'post-throw':
            new_ball_progress = min(ball_progress + 0.08, 1.0)
            
            try:
                current_idx = all_frames.index(current_frame)
                next_idx = current_idx + 1
            except ValueError:
                next_idx = len(all_frames) - 1
            
            if next_idx >= len(all_frames) and new_ball_progress >= 1.0:
                return all_frames[-1] if all_frames else current_frame, True, 'post-throw', 1.0
            
            next_frame = all_frames[next_idx] if next_idx < len(all_frames) else all_frames[-1]
            return next_frame, False, 'post-throw', new_ball_progress
        
        return current_frame, True, phase, ball_progress
    
    # Callback: Handle selection
    @app.callback(
        Output('feedback-panel', 'children'),
        Output('feedback-panel', 'style', allow_duplicate=True),
        Output('user-selection', 'data', allow_duplicate=True),
        Output('selection-correct', 'data', allow_duplicate=True),
        Output('app-phase', 'data', allow_duplicate=True),
        Input('field', 'clickData'),
        State('play-data', 'data'),
        State('app-phase', 'data'),
        prevent_initial_call=True
    )
    def handle_selection(click_data, play_data, phase):
        """Handle receiver selection."""
        hidden_style = {'display': 'none'}
        
        if click_data is None or play_data is None:
            return "", hidden_style, None, None, phase
        
        if phase != 'paused':
            return "", hidden_style, None, None, phase
        
        point = click_data['points'][0]
        if 'customdata' not in point:
            return "", hidden_style, None, None, phase
        
        selected_id = point['customdata']
        
        preds = receivers_app[
            (receivers_app['game_id'] == play_data['game_id']) & 
            (receivers_app['play_id'] == play_data['play_id'])
        ]
        
        selected_row = preds[preds['nfl_id'] == selected_id]
        if len(selected_row) == 0:
            return "", hidden_style, None, None, phase
        
        selected_row = selected_row.iloc[0]
        selected_prob = selected_row['catch_probability']
        selected_route = selected_row.get('predicted_route', '?')
        selected_name = selected_row.get('receiver_name', 'Unknown')
        
        is_best = selected_id == play_data['best_nfl_id']
        
        # Build feedback
        if is_best:
            verdict = "✅ Great read! You found the best option."
            verdict_color = '#4caf50'
        elif selected_prob >= play_data['best_prob'] * 0.8:
            verdict = "👍 Solid choice - close to optimal."
            verdict_color = '#ff9800'
        else:
            verdict = f"❌ Better option available: {play_data['best_route']} ({play_data['best_prob']*100:.0f}%)"
            verdict_color = '#f44336'
        
        # QB's actual choice
        if 'actual_nfl_id' in play_data:
            qb_text = f"QB threw: {play_data.get('actual_route', '?')} ({play_data.get('actual_prob', 0)*100:.0f}%) → {play_data['result']}"
        else:
            qb_text = "QB's target unknown"
        
        feedback = html.Div([
            html.H3(f"You selected: {selected_name}", style={'color': 'white', 'marginBottom': '10px'}),
            html.P(f"Route: {selected_route} | Catch Prob: {selected_prob*100:.0f}%", style={'color': '#ccc'}),
            html.P(verdict, style={'color': verdict_color, 'fontSize': '18px', 'fontWeight': 'bold'}),
            html.Hr(style={'borderColor': '#444'}),
            html.P(qb_text, style={'color': '#aaa'}),
            html.P("Press Play to see the result →", style={'color': '#888', 'marginTop': '15px'})
        ])
        
        show_style = {
            'backgroundColor': '#2a2a2a', 'padding': '20px', 'margin': '20px auto',
            'maxWidth': '600px', 'borderRadius': '10px', 'display': 'block'
        }
        
        return feedback, show_style, selected_id, is_best, 'selected'
    
    return app

Loading data files...
✓ receivers_app: 63,219 rows
✓ plays_app: 13,770 rows
✓ tracking_app: 5,295,049 rows
✓ supplementary_app: 18,009 rows

  Input tracking frames: 4,754,317
  Output tracking frames: 540,732

FEATURE SUMMARY: receivers_app
Shape: 63,219 rows × 17 columns
Memory: 20.87 MB
----------------------------------------------------------------------

game_id
  dtype: int64
  non-null: 63,219 (100.0%)
  min: 2023090700
  max: 2024010713

play_id
  dtype: int64
  non-null: 63,219 (100.0%)
  min: 54
  max: 5258

nfl_id
  dtype: int64
  non-null: 63,219 (100.0%)
  min: 30842
  max: 56663

receiver_name
  dtype: object
  non-null: 63,219 (100.0%)
  unique: 516
  sample: Josh Reynolds, Amon-Ra St. Brown, Brock Wright, Jerick McKinnon, Justin Watson, Marquez Valdes-Scantling, Noah Gray, Skyy Moore, Richie James, Clyde Edwards-Helaire...

receiver_position
  dtype: object
  non-null: 63,219 (100.0%)
  unique: 12
  values: CB, DT, FB, ILB, K, MLB, P, QB, RB, T, TE, WR

predicted_route

In [ ]:
# %% [markdown]
# # Cell 6: Run App

# %%
app = create_app()
app.run(debug=True, port=8050, jupyter_mode='external')

Dash app running on http://127.0.0.1:8050/


---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Cell In[5], line 592, in create_app.<locals>.update_field(
    current_frame=19,
    play_data={'actual_nfl_id': 54476, 'actual_prob': 0.3616452809022951, 'actual_route': 'GO', 'best_nfl_id': 46542, 'best_prob': 0.9847470864714346, 'best_route': 'HITCH', 'concept': 'QUICK', 'game_id': 2023101900, 'input_frames': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], 'output_frames': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...], ...},
    phase='post-throw',
    selected_id=46542,
    selection_correct=True,
    ball_progress=0.4
)
    589 receiver_rankings = play_data.get('receiver_rankings', {})
    591 if len(tracking_frame) > 0:
--> 592     fig = add_players_to_figure(
        fig = Figure({
    'data': [],
    'layout': {'annotations': [{'font': {'color': 'white', 'size': 20},
                              